# Modelo GRU para detección de noticias falsas

### Análisis básico de los datasets de entrenamiento

En este paso, cargamos los dos datasets de entrenamiento disponibles para nuestro problema de clasificación binaria de noticias falsas.

Para cada dataset mostramos:

- Información general (número de filas, tipos de datos, valores nulos)
- Distribución relativa de la variable objetivo `label` (balance entre clases)
- Estadísticas básicas de la longitud del texto de la columna `statement` (mínimo, máximo, media, percentiles)

Esto nos ayudará a decidir cuál dataset es más adecuado para entrenar nuestro modelo GRU.


In [32]:
import pandas as pd

In [33]:
# Cargar datasets de entrenamiento
train_preprocess = pd.read_csv('train_simp_preprocess_v2.csv')
train_limpieza = pd.read_csv('train_simp_limpieza_v2.csv')

# Mostrar información básica y distribución clases train_preprocess
print("== train_simp_preprocess_v2.csv ==")
print(train_preprocess.info())
print("\nDistribución de clases (label):")
print(train_preprocess['label'].value_counts(normalize=True))
print("\nEstadísticas longitud de textos (statement):")
print(train_preprocess['statement'].str.len().describe())

print("\n== train_simp_limpieza_v2.csv ==")
print(train_limpieza.info())
print("\nDistribución de clases (label):")
print(train_limpieza['label'].value_counts(normalize=True))
print("\nEstadísticas longitud de textos (statement):")
print(train_limpieza['statement'].str.len().describe())


== train_simp_preprocess_v2.csv ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  8950 non-null   object
 1   label                               8950 non-null   int64 
 2   statement                           8950 non-null   object
 3   subject                             8950 non-null   object
 4   speaker                             8950 non-null   object
 5   speaker_job                         8950 non-null   object
 6   state_info                          8950 non-null   object
 7   party_affiliation                   8950 non-null   object
 8   party_affiliation_uni               8950 non-null   object
 9   party_affiliation_category_map      8950 non-null   object
 10  statement_tokens                    8950 non-null   object
 11  num_tokens           

### Preprocesamiento del texto

Antes de entrenar el modelo GRU, necesitamos preparar el texto de entrada para que sea compatible con el modelo.

Pasos a realizar:

- Normalización del texto (minúsculas y limpieza básica si es necesario)  
- Tokenización usando `Tokenizer` de Keras para convertir palabras a índices enteros  
- Padding para que todas las secuencias tengan la misma longitud, necesaria para procesar en batches en el modelo  

Esto nos permitirá transformar las oraciones en secuencias numéricas que el modelo podrá entender.


In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Cargar dataset elegido
train = pd.read_csv('train_simp_limpieza_v2.csv')

# Inspección rápida para comprobar la columna
print(train['statement'].head())

# Normalizar texto: pasamos a minúsculas (opcional se puede limpiar más)
train['statement'] = train['statement'].str.lower()

# Parámetros para tokenización
max_words = 10000   # máximo vocabulario
max_len = 150       # longitud máxima de secuencia (ajustable)

# Crear el tokenizer y ajustarlo al texto
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train['statement'])

# Convertir textos a secuencias de enteros
train_sequences = tokenizer.texts_to_sequences(train['statement'])

# Padding para igualar la longitud de las secuencias
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')

# Etiquetas
train_labels = train['label'].values

# Mostrar ejemplo de secuencia tokenizada y padded
print("Ejemplo texto:", train['statement'].iloc[0])
print("Secuencia tokenizada:", train_sequences[0])
print("Secuencia padded:", train_padded[0])
print("Etiqueta:", train_labels[0])


0    China is in the South China Sea and (building)...
1    With the resources it takes to execute just ov...
2    The (Wisconsin) governor has proposed tax give...
3    Says her representation of an ex-boyfriend who...
4    At protests in Wisconsin against proposed coll...
Name: statement, dtype: object
Ejemplo texto: china is in the south china sea and (building)a military fortress the likes of which perhaps the world has not seen.
Secuencia tokenizada: [318, 11, 3, 2, 657, 318, 2536, 7, 777, 6, 295, 6487, 2, 3289, 4, 179, 2537, 2, 165, 13, 30, 522]
Secuencia padded: [ 318   11    3    2  657  318 2536    7  777    6  295 6487    2 3289
    4  179 2537    2  165   13   30  522    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    

### Construcción y entrenamiento del modelo GRU

En esta etapa construimos un modelo secuencial con las siguientes capas:

- Embedding: para convertir índices de palabras en vectores densos de tamaño fijo  
- GRU: capa recurrente que captura dependencias temporales en la secuencia  
- Dense: capa densa final con activación sigmoid para clasificación binaria  

Después compilamos el modelo con pérdida `binary_crossentropy`, optimizador Adam y medimos precisión.

Finalmente, entrenamos el modelo usando validación interna para monitorear performance.


In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

# Variables globales de tokenización que definiste
max_words = 10000
max_len = 150

embedding_dim = 64
gru_units = 64

# Crear el modelo asegurando que el input_length y input_dim están bien definidos
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    GRU(gru_units, return_sequences=False),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Forzar construcción del modelo para que se muestre correctamente el summary
model.build(input_shape=(None, max_len))

# Ahora muestra el resumen con los parámetros
model.summary()

# Compilar el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 150, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 665,025 (2.54 MB)

 Trainable params: 665,025 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
%pip install numpy scipy scikit-learn tensorflow matplotlib pandas
